In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-08 11:39:00,2922.36,2923.29,2922.07,2923.07,19.0,0
1,2024-11-08 11:40:00,2923.04,2925.72,2922.25,2925.06,41.0,1
2,2024-11-08 11:41:00,2925.05,2927.09,2924.71,2927.09,20.0,2
3,2024-11-08 11:42:00,2926.76,2926.97,2925.40,2926.19,69.0,3
4,2024-11-08 11:43:00,2926.14,2926.15,2922.22,2922.44,82.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

In [8]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,NaN,NaN,0.02,0
1,2892.600000,NaN,0.04,0
2,2893.924800,NaN,0.06,0
3,2895.914712,NaN,0.06,0
4,2897.785229,NaN,0.06,0
...,...,...,...,...
345,2895.506661,NaN,0.04,0
346,2895.876795,NaN,0.04,0
347,2896.232123,NaN,0.04,0
348,NaN,2904.760,0.02,1


In [9]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [10]:
df.head()

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,psar_flip_dir
0,2024-11-08 11:39:00,2922.36,2923.29,2922.07,2923.07,19.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,0
1,2024-11-08 11:40:00,2923.04,2925.72,2922.25,2925.06,41.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2892.600000,NaN,0.04,0,0
2,2024-11-08 11:41:00,2925.05,2927.09,2924.71,2927.09,20.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2893.924800,NaN,0.06,0,0
3,2024-11-08 11:42:00,2926.76,2926.97,2925.40,2926.19,69.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2895.914712,NaN,0.06,0,0
4,2024-11-08 11:43:00,2926.14,2926.15,2922.22,2922.44,82.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2897.785229,NaN,0.06,0,0


In [11]:
df['r_ema_s_m'] = df['EMA_10'] / df['EMA_50']

In [13]:
len_data = 5

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_10', len_data)
    df.at[i + len_data-1, 'gradient_ema_10'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_14', len_data)
    df.at[i + len_data-1, 'gradient_rsi_14'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient

In [14]:
mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
df['flag_ema_crossing'] = 0
df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [15]:
mask_ema_grad_pos = (df['gradient_ema_10']>0.05)
mask_ema_grad_neg = (df['gradient_ema_10']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_14']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_14']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [17]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_10']>df['EMA_50']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_10']<df['EMA_50']), 'ema_short_above_or_below'] = -1

In [20]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_10'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_10'] / df['BBL_20_2.0']

In [21]:
df['long_entry'] = np.nan
df['short_entry'] = np.nan
df['long_exit'] = np.nan
df['short_exit'] = np.nan

In [67]:
# display(df.iloc[100:120])

# Strategy

To create a real-time trading strategy with specific technical indicator settings and strategies for entering and exiting trades, let's break down the components clearly:

### 1. **Technical Indicators**
Here's how we can set up the specific technical indicators:

#### Moving Averages
- **Short-Term Moving Average (MA)**: Use a 10-period MA. This is effective for spotting short-term trends and suitable for minute-by-minute trading data.
- **Long-Term Moving Average**: Use a 50-period MA to confirm the overall trend direction. This helps filter out the noise and provides a clearer direction.

#### Relative Strength Index (RSI)
- **Length**: A 14-period RSI is standard and works well in a variety of market conditions.
- **Overbought Threshold**: Set at 70 to indicate potential selling points.
- **Oversold Threshold**: Set at 30 to indicate potential buying points.

#### MACD (Moving Average Convergence Divergence)
- **Fast Line**: 12-period Exponential Moving Average (EMA)
- **Slow Line**: 26-period EMA
- **Signal Line**: 9-period EMA of the MACD line itself

### 2. **Entry and Exit Points**
- **Go Long (Buy)**: 
  - **Entry Point**: When the 10-period MA crosses above the 50-period MA and the RSI is below 70 but above 30, indicating neither overbought nor oversold conditions. Additionally, the MACD line should cross above the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back below the 50-period MA, or RSI approaches 70, indicating a potential overbought condition.
  
- **Go Short (Sell)**: 
  - **Entry Point**: When the 10-period MA crosses below the 50-period MA and the RSI is above 30 but below 70, indicating normal conditions. Also, the MACD line should cross below the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back above the 50-period MA, or RSI approaches 30, indicating a potential oversold condition.

### 3. **Stop Loss Criteria**
- **For Long Positions**: Place a stop loss below the most recent significant low or a fixed percentage, such as 2-3% below the entry price, depending on your risk tolerance.
- **For Short Positions**: Place a stop loss above the most recent significant high or a fixed percentage, such as 2-3% above the entry price.

### 4. **Risk Management**
- Consider the amount of capital at risk per trade, commonly 1-2% of your total trading capital to manage risk effectively.
- Use trailing stops to lock in profits as the market moves in your favor.

### 5. **Automation and Monitoring**
- Automate the strategy using a trading bot if possible, especially to handle the high-frequency nature of minute-by-minute trading.
- Regularly monitor performance and adjust parameters as necessary based on market conditions and strategy performance.

This setup gives you a robust framework for trading on a minute-by-minute basis, leveraging these technical indicators to make informed trading decisions. Adjust these settings based on backtesting results and your specific risk tolerance.

# Long Entry

In [25]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)
mask_le2 = (df['MACDh_12_26_9']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_14']<70)&(df['RSI_14']>30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close'])

In [26]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_le4, 'rsi_30_to_70'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [27]:
# df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'close_to_bbl', 'rsi_30_to_50', 'PSAR_bellow_close']].sum(axis=1)

In [28]:
df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'rsi_30_to_70', 'PSAR_bellow_close']].sum(axis=1)

In [45]:
df.loc[df['long_entry'] == 4].shape

(7, 48)

# Short Entry

In [54]:
mask_se1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_se2 = (df['MACDh_12_26_9']<0)
mask_se3 = (df['r_close_bbu']>=0.9995)
mask_se4 = (df['RSI_14']>30)&(df['RSI_14']<70)
mask_se5 = (df['PSARs_0.02_0.2'] > df['close'])

In [55]:
df['ema_crossing_neg'] = 0
df.loc[mask_se1, 'ema_crossing_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_se2, 'macd_neg'] = 1
df['close_to_bbu'] = 0
df.loc[mask_se3, 'close_to_bbu'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_se4, 'rsi_30_to_70'] = 1
df['PSAR_above_close'] = 0
df.loc[mask_se5, 'PSAR_above_close'] = 1

In [56]:
# df['short_entry'] = df[['ema_crossing_neg', 'macd_neg', 'close_to_bbu', 'rsi_50_to_70', 'PSAR_above_close']].sum(axis=1)

In [58]:
df['short_entry'] = df[['ema_crossing_neg', 'macd_neg', 'rsi_30_to_70', 'PSAR_above_close']].sum(axis=1)

In [60]:
df.loc[df['short_entry']==4].shape

(22, 51)

# Long Exit

In [32]:
mask_lex1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_lex2 = (df['RSI_14']>=70)
mask_lex3 = (df['psar_flip_dir'] == -1)

In [33]:
df['ema_crossing_neg'] = 0
df.loc[mask_lex1, 'ema_crossing_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1

In [49]:
df['long_exit'] = df[['ema_crossing_neg', 'rsi_above_70']].sum(axis=1)

In [73]:
df.loc[df['long_exit'] == 2].shape

(0, 57)

# Short Exit

In [63]:
mask_sex1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)
mask_sex2 = (df['RSI_14']<=30)
mask_sex3 = (df['psar_flip_dir'] == 1)

In [64]:
df['ema_crossing_pos'] = 0
df.loc[mask_sex1, 'ema_crossing_pos'] = 1
df['rsi_bellow_30'] = 0
df.loc[mask_sex2, 'rsi_bellow_30'] = 1
df['psar_flip_pos'] = 0
df.loc[mask_sex3, 'psar_flip_pos'] = 1

In [65]:
df['short_exit'] = df[['ema_crossing_pos', 'rsi_bellow_30']].sum(axis=1)

In [66]:
df.loc[df['short_exit'] == 1].shape

(33, 54)

# Bullish / bearish confirmation

In [70]:
df['gradient_ema_pos'] = 0
df.loc[df['flag_grad_ema']==1, 'gradient_ema_pos'] = 1
df['gradient_ema_neg'] = 0
df.loc[df['flag_grad_ema']==-1, 'gradient_ema_neg'] = 1

In [71]:
df['bullish_confirm'] = df[['ema_crossing_pos', 'macd_pos', 'gradient_ema_pos']].sum(axis=1)
df['bearish_confirm'] = df[['ema_crossing_neg', 'macd_neg', 'gradient_ema_neg']].sum(axis=1)

# Oversold Marker

In [74]:
mask_os1 = (df['RSI_14']<=30)&((df['flag_grad_ls']==1)|(df['flag_grad_ema']==1))
mask_os2 = (df['r_close_bbl']<=1.0005)&((df['flag_grad_ls']==1)|(df['flag_grad_ema']==1))

In [75]:
df['rsi_bellow_30'] = 0
df.loc[mask_os1, 'rsi_bellow_30'] = 1
df['close_to_bbl'] = 0
df.loc[mask_os2, 'close_to_bbl'] = 1

In [76]:
df['oversold_confirm'] = df[['rsi_bellow_30', 'close_to_bbl']].sum(axis=1)

# Overbought Marker

In [78]:
mask_ob1 = (df['RSI_14']>=70)&((df['flag_grad_ls']==-1)|(df['flag_grad_ema']==-1))
mask_ob2 = (df['r_close_bbu']>=0.9995)&((df['flag_grad_ls']==-1)|(df['flag_grad_ema']==-1))

In [79]:
df['rsi_above_70'] = 0
df.loc[mask_ob1, 'rsi_above_70'] = 1
df['close_to_bbu'] = 0
df.loc[mask_ob2, 'close_to_bbu'] = 1

In [80]:
df['overbought_confirm'] = df[['rsi_above_70', 'close_to_bbu']].sum(axis=1)

In [81]:
df.to_clipboard()